## Wstęp

Algorytm McEliece to jedna z pierwszych propozycji kryptosystemu klucza publicznego, który został wprowadzony w 1978 roku przez Roberta McEliece'a. Algorytm McEliece jest interesujący, ponieważ jest jednym z nielicznych systemów klucza publicznego, które opierają swoje bezpieczeństwo na twardości problemów z kodowania korekcji błędów, co czyni go odpornym na ataki przez komputery kwantowe.

System kryptograficzny McEliece jest asymetrycznym algorytmem opartym na problemie kodów korekcyjnych. System ten może być wykorzystywany do przesyłania w bezpieczny sposób klucza symetrycznego.

Posiada 3 podstawowe parametry
n: długość kodu, określa liczbę bitów w zaszyfrowanym tekscie.
k: ilość bitów w tekście jawnym
t: zdolność do korekcji błędów kodu.
m: definiuje rozmiar ciała skończonego

Przykłady wartości parametrów:

oryginalne wartości:
n = 1024, k = 524, t = 101, m = 50

aktualnie zalecane wartości:
n = 2048, k = 1751, t = 27, m = 27

postkwantowe wartości:
n = 6960, k = 5413, t = 119, m = 13
n = 8192, k = 6528, t = 128, m = 13


In [58]:
# m = 13
# n = 6960
# q = 8192
# t = 119
# k = 5413

# m = 50
# n = 1024
# q = 1024
# t = 50
# k = 524

# test params
m = 4
n = 16
q = 16
t = 2
k = 8

Algorytm składa się z 3 głównym kroków:

* generowanie kluczy
* enkodowanie
* dekodowanie

## Generowanie kluczy

Generowanie kluczy składa się z 7 kroków:
1 Generuje losowy moniczny wielomian nierozkładalny g(x) o stopniu t w ciele galoisa.
2 Wybiera losowy ciąg n różnych elementów ciała skończonego Fq.
3 Oblicza macierz H ̃ o wymiarach t×n.
4 Tworzy macierz Hˆ o wymiarach mt × n, zamieniając elementy macierzy H ̃ na odpowiednie kolumny bitów.
5 Stosujemy eliminację Gaussa na macierzy Hˆ, aby przekształcić ją do postaci systematycznej (In−k | T).
6 Generuje losowy ciąg bitów s o długości n.
7 Tworzy klucz prywatny (s,Γ) i klucz publiczny T.


In [59]:
%pip install galois

Note: you may need to restart the kernel to use updated packages.


In [60]:
%pip install sympy

Note: you may need to restart the kernel to use updated packages.


In [61]:
import galois
import numpy as np
import random

In [62]:
#1
def generate_irreducable_poly(degree):
    return galois.irreducible_poly(order=q, degree=degree, method="random")

In [63]:
poly = generate_irreducable_poly(t)
poly

Poly(x^2 + 8x + 15, GF(2^4))

In [64]:
#2
def generate_distinct_elements(n, q):
    return np.random.choice(np.arange(q), size=n, replace=False)

In [65]:
random_el = generate_distinct_elements(n, q)
random_el

array([12,  7,  8,  2,  6,  3, 15,  5, 14, 11,  4,  9, 13, 10,  1,  0])

Compute the t × n matrix H ̃ = h[i,j] over Fq, where h[i,j] = α[i−1]/g(α[j]) for i = 1,...,t and j = 1,...,n.

In [66]:
#3
def calculate_matrix(alpha, g, t, n, m):
    GF = galois.GF(2 ** m)
    g = galois.Poly(g.coeffs, field=GF)

    H = np.empty((t, n), dtype=object)

    for i in range(1, t + 1):
        for j in range(1, n + 1):
            alpha_j_GF = GF(alpha[j - 1])
            g_alpha_j = g(alpha_j_GF)
            H[i - 1, j - 1] = alpha_j_GF ** (i - 1) / g_alpha_j

    return H

In [67]:
matrix = calculate_matrix(random_el, poly, t, n, m)
matrix

array([[GF(12, order=2^4), GF(13, order=2^4), GF(8, order=2^4),
        GF(15, order=2^4), GF(4, order=2^4), GF(1, order=2^4),
        GF(13, order=2^4), GF(14, order=2^4), GF(4, order=2^4),
        GF(1, order=2^4), GF(12, order=2^4), GF(7, order=2^4),
        GF(14, order=2^4), GF(15, order=2^4), GF(7, order=2^4),
        GF(8, order=2^4)],
       [GF(15, order=2^4), GF(5, order=2^4), GF(12, order=2^4),
        GF(13, order=2^4), GF(11, order=2^4), GF(3, order=2^4),
        GF(7, order=2^4), GF(3, order=2^4), GF(13, order=2^4),
        GF(11, order=2^4), GF(5, order=2^4), GF(10, order=2^4),
        GF(10, order=2^4), GF(12, order=2^4), GF(7, order=2^4),
        GF(0, order=2^4)]], dtype=object)

In [68]:
#4
def bits_in_GF2(element, t):
    return [int(b) for b in format(element, f'0{t}b')]


def transform_to_H_hat(H, m):
    rows, cols = H.shape
    H_hat = np.empty((m * rows, cols), dtype=int)

    for i in range(rows):
        for j in range(cols):
            H_hat[m * i: m * (i + 1), j] = bits_in_GF2(H[i, j], m)

    return H_hat

In [69]:
transformed = transform_to_H_hat(matrix, m)
transformed

array([[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1],
       [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0],
       [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0],
       [1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0]])

In [70]:
#5
def gauss_elimination(matrix, m, t):
    gf2 = galois.GF(2)
    matrix = gf2(matrix)

    rows, cols = matrix.shape
    for i in range(rows):
        if matrix[i, i] == 0:
            for j in range(i+1, rows):
                if matrix[j, i] == 1:
                    matrix[[i, j]] = matrix[[j, i]]
                    break
        for j in range(rows):
            if i != j and matrix[j, i] == 1:
                matrix[j] ^= matrix[i]

    I = np.eye(m * t, dtype=int)
    if np.array_equal(matrix[:m * t, :m * t], I):
        I = matrix[:, :m * t]
        T = matrix[:, m * t:]
        return np.hstack((I, T)), T
    else:
        return None

In [71]:
gauss = gauss_elimination(transformed, m, t)
gauss[0]

GF([[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1]], order=2)

In [72]:
gauss[1]

GF([[1, 1, 0, 1, 1, 0, 1, 0],
    [0, 0, 1, 0, 1, 1, 0, 1],
    [0, 1, 1, 1, 0, 1, 1, 0],
    [1, 0, 0, 1, 0, 1, 0, 1],
    [1, 1, 1, 0, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 1, 0, 0],
    [1, 0, 1, 1, 0, 0, 1, 0],
    [1, 0, 0, 0, 1, 0, 1, 1]], order=2)

In [73]:
#6
def generate_random_string(n):
    s = "{0:b}".format(random.getrandbits(n))
    s = s.zfill(n)
    return s

In [74]:
generate_random_string(n)

'0111001010000110'

In [75]:
def get_keys():
    counter = 0
    while True:
        counter += 1
        print(f"Iteration {counter}")
        poly = generate_irreducable_poly(t)
        random_seq = generate_distinct_elements(n, q)
        matrix = calculate_matrix(random_seq, poly, t, n, m)
        H_hat = transform_to_H_hat(matrix, m)

        T = gauss_elimination(H_hat, m, t)
        if T is not None:
            print("Gaussian elimination successful!")
            print(T)
            s = generate_random_string(n)

            #7
            Γ = (s, (poly, random_seq))
            print(f"Private key:( \n{s}, \n{Γ})")
            print(f"Public key: \n{T}")
            return Γ, T
        else:
            print("Gaussian elimination failed, retrying...")

In [76]:
keys = get_keys()

Iteration 1
Gaussian elimination successful!
(GF([[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1],
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]], order=2), GF([[1, 0, 1, 0, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 0, 1, 1],
    [0, 1, 1, 1, 1, 0, 0, 1],
    [0, 1, 0, 0, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 0, 1, 1, 1, 1, 1],
    [0, 1, 1, 1, 0, 0, 1, 0]], order=2))
Private key:( 
0001111000011000, 
('0001111000011000', (Poly(x^2 + 7x + 9, GF(2^4)), array([13, 11,  6,  0, 14,  9, 15, 10,  4,  2,  3,  5,  8, 12,  1,  7]))))
Public key: 
(GF([[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1],
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 

## Enkodowanie

1 Tworzymy wektor błędów e, ma on się składać z dokładnie t wystąpień 1, i reszty 0
2 Posiadając macierz T (klucz publiczny) odtwarzamy macierz H poprzez dodanie do niej macierzy jednostowej
3 mnożymy macierz H*e

In [77]:
def createH(T, m, t):
    I = np.eye(m * t, dtype=int)
    return np.concatenate((I, T), axis=1)

In [78]:
keys[1][1]

GF([[1, 0, 1, 0, 1, 0, 0, 1],
    [1, 1, 0, 0, 0, 0, 1, 1],
    [0, 1, 1, 1, 1, 0, 0, 1],
    [0, 1, 0, 0, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 0],
    [1, 1, 0, 1, 0, 1, 0, 1],
    [1, 0, 0, 1, 1, 1, 1, 1],
    [0, 1, 1, 1, 0, 0, 1, 0]], order=2)

In [79]:
H = createH(keys[1][1], m, t)
H

GF([[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1],
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1],
    [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]], order=2)

In [80]:
global k, n, t

gf2 = galois.GF(2)
def encrypt(message, public_key):
    if len(message) != k:
        print(f"McEliece.encrypt: The message must be of length {k}")
        return
    error = np.zeros(n, dtype=int)
    error[0:t] = 1
    np.random.shuffle(error)

    ciphertext = gf2(message).dot(gf2(public_key)) + gf2(error)
    return ciphertext, error

In [81]:
message = [1,0,1,0,1,0,1,0]
encrypted = encrypt(message, H)
encrypted

(GF([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0], order=2),
 array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]))

## Dekodowanie

1. Rozszerzanie C: Na początku, rozszerzamy wektor C przez dodanie do niego k zer, gdzie k to długość klucza prywatnego. Nowy wektor jest oznaczony jako v
2. Wyszukiwanie kodu: Następnie szukamy unikalnego kodu w kodzie Goppa zdefiniowanym przez Γ, który jest w odległości ≤ t od v. Jeżeli nie ma takiego kodu, to zwracamy failure (⊥)
3. Ustawianie e: Po znalezieniu kodu c, tworzymy nowy wektor e przez dodanie (operacja XOR) v i c
4. Sprawdzanie warunków: Sprawdzamy dwa warunki:
    - Czy waga (liczba niezerowych bitów) e wynosi t
    - Czy C0 równa się He, gdzie H jest macierzą generującą kodu, tworzoną na podstawie klucza prywatnego

In [ ]:
# TODO
def get_unique_goppa_code(v, alphas, t, poly):
    return

In [83]:
def decode(C, s, poly, alphas, t, n, k, public_key):
    v = np.append(C, [0]*k)

    c = get_unique_goppa_code(v, alphas, t, poly)
    if c is None:
        return "⊥"

    e = np.bitwise_xor(v, c)
    if np.count_nonzero(e) == t and np.array_equal(C, np.dot(public_key, e) % 2):
        return e
    else:
        return "⊥"